# Importing Necessary Libraries

In [ ]:
!pip install nltk
!pip install wordlist

In [ ]:
import os
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
nltk.download('stopwords')


# Defining The Keyword Dictionary

In [ ]:
word_lists = {
    "Catastrophe": [
        'operation', 'event', 'disaster', 'business', 'natural', 'result',
        'facility', 'loss', 'damage', 'disruption', 'system', 'weather',
        'failure', 'condition', 'including', 'adversely', 'terrorist',
        'affect', 'attack', 'risk', 'act', 'interruption', 'earthquake',
        'impact', 'adverse', 'hurricane', 'significant', 'fire',
        'financial', 'company'
    ],
    "Corporate_Governance": [
        'stock', 'stockholder', 'director', 'share', 'common', 'board',
        'provision', 'control', 'shareholder', 'right', 'change', 'company',
        'law', 'voting', 'preferred', 'incorporation', 'bylaw', 'certain',
        'prevent', 'class', 'approval', 'outstanding', 'interest',
        'transaction', 'price', 'business', 'vote', 'ownership', 'holder',
        'certificate'
    ],
    "Country": [
        'rate', 'currency', 'interest', 'foreign', 'risk', 'exchange', 'result',
        'dollar', 'fluctuation', 'change', 'financial', 'operation', 'market',
        'value', 'increase', 'affect', 'exposure', 'income', 'asset', 'company',
        'impact', 'investment', 'revenue', 'net', 'hedging', 'adversely',
        'hedge', 'contract', 'price', 'loss'
    ],
    "Customer_Concentration": [
        'sale', 'revenue', 'customer', 'approximately', 'company', 'product',
        'year', 'agreement', 'united', 'state', 'service', 'operation',
        'business', 'significant', 'portion', 'fiscal', 'market', 'facility',
        'net', 'store', 'december', 'total', 'international', 'contract',
        'outside', 'including', 'number', 'accounted', 'located', 'ended'
    ],
    "Economic_Conditions": [
        'economic', 'business', 'foreign', 'risk', 'operation', 'condition',
        'market', 'change', 'including', 'country', 'political', 'international',
        'subject', 'state', 'law', 'product', 'result', 'financial', 'rate', 'united',
        'regulation', 'factor', 'currency', 'control', 'cost', 'tax', 'affect',
        'government', 'adversely', 'trade'
    ],
    "Energy_Sector": [
        'gas', 'price', 'natural', 'oil', 'cost', 'operation', 'energy', 'result',
        'production', 'market', 'demand', 'power', 'company', 'fuel', 'customer',
        'drilling', 'including', 'increase', 'facility', 'service', 'regulation',
        'emission', 'change', 'coal', 'risk', 'supply', 'condition', 'new',
        'business', 'future'
    ],
    "Financing_I": [
        'company', 'financial', 'insurance', 'investment', 'market', 'risk',
        'credit', 'capital', 'rating', 'bank', 'business', 'security', 'fund',
        'institution', 'including', 'asset', 'certain', 'service', 'subsidiary',
        'result', 'loss', 'regulation', 'subject', 'rate', 'regulatory',
        'requirement', 'federal', 'ability', 'state', 'liquidity'
    ],
    "Financing_II": [
        'credit', 'facility', 'debt', 'note', 'agreement', 'million', 'senior',
        'indebtedness', 'covenant', 'certain', 'default', 'interest', 'asset',
        'amount', 'subsidiary', 'outstanding', 'financial', 'ability', 'payment',
        'secured', 'december', 'event', 'loan', 'term', 'company', 'stock',
        'revolving', 'obligation', 'including', 'make'
    ],
    "Financing_III": [
        'capital', 'debt', 'ability', 'cash', 'business', 'credit', 'financing',
        'additional', 'operation', 'future', 'fund', 'financial', 'market',
        'indebtedness', 'term', 'condition', 'flow', 'obligation', 'available',
        'affect', 'obtain', 'result', 'adversely', 'make', 'company', 'interest',
        'need', 'acquisition', 'able', 'service'
    ],
    "Growth_and_Restructuring": [
        'business', 'acquisition', 'operation', 'company', 'acquired', 'risk',
        'result', 'cost', 'management', 'financial', 'future', 'growth',
        'including', 'product', 'benefit', 'investment', 'new', 'significant',
        'successfully', 'strategy', 'system', 'difficulty', 'anticipated',
        'strategic', 'venture', 'resource', 'technology', 'time', 'integration',
        'able'
    ],

    "Healthcare_Spending": [
        "program", "state", "service", "health", "government", "federal",
        "care", "contract", "medicare", "payment", "law", "healthcare",
        "regulation", "cost", "act", "change", "including", "provider",
        "result", "business", "certain", "medicaid", "reimbursement",
        "rate", "revenue", "patient", "hospital", "year", "subject",
        "insurance"
    ],
    "Human_Capital": [
        "personnel", "business", "retain", "ability", "key", "employee",
        "attract", "management", "qualified", "success", "service",
        "new", "executive", "operation", "depends", "future", "company",
        "adversely", "officer", "growth", "result", "senior", "loss",
        "able", "upon", "sale", "unable", "store", "highly", "competition"
    ],
    "Incomplete_Contracts": [
        "agreement", "company", "director", "certain", "court", "officer",
        "action", "state", "stock", "board", "filed", "law", "executive",
        "subject", "interest", "merger", "business", "including", "share",
        "inc", "party", "shareholder", "district", "right", "claim",
        "security", "term", "transaction", "addition", "time"
    ],
    "Information_Systems": [
        "system", "information", "business", "security", "customer",
        "service", "data", "result", "operation", "breach", "technology",
        "failure", "party", "loss", "financial", "risk", "reputation",
        "including", "third", "company", "network", "disruption",
        "damage", "product", "computer", "significant", "ability",
        "employee", "client", "adversely"
    ],
    "Intangible_Assets": [
        "asset", "value", "impairment", "result", "goodwill", "future",
        "intangible", "charge", "estimate", "fair", "million", "loss",
        "change", "financial", "net", "cost", "significant", "amount",
        "contract", "operation", "investment", "period", "cash",
        "carrying", "assumption", "market", "required", "company",
        "revenue", "december"
    ],
    "Intellectual_Property": [
        "right", "property", "intellectual", "patent", "product",
        "technology", "party", "protect", "proprietary", "license",
        "third", "claim", "trademark", "business", "infringement",
        "use", "agreement", "trade", "others", "litigation", "secret",
        "protection", "company", "result", "law", "certain", "obtain",
        "future", "service", "addition"
    ],
    "Product_Approval": [
        "product", "approval", "fda", "regulatory", "clinical", "market",
        "development", "drug", "trial", "sale", "result", "new",
        "including", "candidate", "company", "marketing", "manufacturing",
        "use", "device", "regulation", "subject", "medical", "state",
        "process", "obtain", "certain", "patient", "pharmaceutical",
        "requirement", "business"
    ],
    "Product_Defects": [
        "claim", "insurance", "liability", "result", "business",
        "litigation", "coverage", "product", "cost", "loss", "financial",
        "operation", "damage", "significant", "future", "risk", "time",
        "company", "legal", "subject", "proceeding", "amount",
        "adverse", "management", "material", "action", "substantial",
        "matter", "lawsuit", "property"
    ],
    "Product_Innovation": [
        "product", "new", "service", "market", "customer", "technology",
        "competitor", "business", "company", "ability", "competitive",
        "industry", "develop", "development", "resource", "change",
        "sale", "result", "financial", "compete", "revenue", "existing",
        "competition", "system", "marketing", "future", "continue",
        "greater", "software", "including"
    ],
    "Real_Estate": [
        "loan", "property", "real", "estate", "loss", "market", "mortgage",
        "condition", "result", "rate", "lease", "increase", "risk",
        "value", "tenant", "interest", "economic", "ability", "adversely",
        "credit", "portfolio", "cost", "business", "affect", "financial",
        "investment", "company", "home", "including", "change"
    ],
    "Regulatory_Change": [
        "regulation", "law", "business", "result", "operation", "subject",
        "change", "regulatory", "cost", "comply", "compliance",
        "state", "financial", "requirement", "government", "affect",
        "future", "adversely", "penalty", "new", "federal", "company",
        "adverse", "effect", "impact", "including", "applicable",
        "environmental", "fine", "failure"
    ],
    "Regulatory_Compliance": [
        "law", "regulation", "environmental", "state", "subject",
        "property", "hazardous", "federal", "liability", "operation",
        "product", "material", "cost", "substance", "including",
        "safety", "local", "certain", "waste", "act", "use", "disposal",
        "health", "facility", "company", "damage", "requirement",
        "various", "site", "contamination"
    ],
    "Reporting_Accuracy": [
        "risk", "statement", "result", "financial", "factor", "report",
        "forward", "looking", "information", "business", "operation",
        "uncertainty", "see", "condition", "item", "form", "discussion",
        "future", "note", "management", "annual", "company",
        "described", "materially", "additional", "following", "actual",
        "currently", "consolidated", "analysis"
    ],


    "Reporting_compliance": [
        "financial", "control", "internal", "reporting", "accounting", "result", "statement",
        "material", "report", "company", "standard", "management", "change", "effective", "weakness",
        "system", "procedure", "act", "required", "public", "stock", "future", "business",
        "sarbanesoxley", "operation", "section", "sec", "disclosure", "maintain", "requirement"
    ],


    "Supply_Chain": [
        "product", "customer", "cost", "material", "supplier", "result", "supply", "manufacturing",
        "component", "contract", "price", "increase", "raw", "delay", "production", "business",
        "ability", "time", "order", "demand", "service", "purchase", "company", "manufacturer",
        "certain", "operation", "adversely", "significant", "new", "sale"
    ],

    "Tax_Uncertainty": [
        "tax", "income", "reit", "change", "rate", "subject", "federal", "taxable", "law", "would",
        "result", "certain", "state", "distribution", "year", "asset", "liability", "company", "future",
        "jurisdiction", "qualify", "net", "foreign", "amount", "provision", "transaction", "revenue",
        "authority", "dividend", "effective"],


    "Volatile_Costs": [
        "plan", "cost", "rate", "pension", "benefit", "increase", "asset", "change", "result", "future",
        "employee", "company", "expense", "obligation", "million", "return", "assumption", "funding",
        "liability", "contribution", "requirement", "significant", "market", "including", "required",
        "interest", "operation", "certain", "year", "financial"
    ],
    "Volatile_Net_Cash_Flows": [
        "result", "financial", "operation", "condition", "business", "adverse", "adversely", "effect",
        "material", "affect", "cash", "impact", "flow", "materially", "operating", "customer",
        "affected", "company", "market", "significant", "product", "risk", "loss", "future",
        "cost", "failure", "economic", "position", "revenue", "would"
    ],

    "Volatile_Revenue": [
        "result", "product", "customer", "sale", "revenue", "price", "market", "operating", "business",
        "demand", "cost", "industry", "adversely", "increase", "condition", "economic", "change",
        "affect", "margin", "future", "quarter", "impact", "significant", "consumer", "service", "level",
        "factor", "decline", "period", "operation"
    ],

    "Volatile_Stock_Price": [
        "stock", "common", "price", "market", "share", "security", "result", "company", "future",
        "operating", "trading", "sale", "decline", "factor", "fluctuation", "change", "affect",
        "financial", "analyst", "equity", "adversely", "dividend", "condition", "investor",
        "performance", "significant", "stockholder", "volatility", "value", "investment"]



}

# Preprocessing function to clean the text

In [ ]:

def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Function to calculate TF-IDF scores for a given text

In [ ]:

def calculate_tfidf(text, word_lists):
    all_words = list(set([word for sublist in word_lists.values() for word in sublist]))
    vectorizer = TfidfVectorizer(vocabulary=all_words)
    tfidf_matrix = vectorizer.fit_transform([text])
    tfidf_scores = pd.DataFrame(tfidf_matrix.T.toarray(), index=vectorizer.get_feature_names_out(), columns=["tfidf"])
    return tfidf_scores

# Function to process each yearwise data folder and compute TF-IDF values

In [ ]:

def process_yearwise_data(dataset_path, word_lists):
    final_results = []
    for folder in os.listdir(dataset_path):
        full_path = os.path.join(dataset_path, folder)
        if os.path.isdir(full_path) and 'combined_yearly_data' in folder:
            print(f"Processing folder: {folder}")
            for file in os.listdir(full_path):
                if file.endswith(".csv"):
                    file_path = os.path.join(full_path, file)
                    year_match = re.match(r'(\d{4})\.csv', file)
                    if year_match:
                        year = year_match.group(1)
                    else:
                        print(f"Year not found in filename: {file}")
                        continue
                    try:
                        df = pd.read_csv(file_path)
                        print(f"Processing file: {file_path}")
                        for index, row in df.iterrows():
                            transcript = row['Extracted 1A']
                            ticker = row['Ticker']
                            cleaned_transcript = preprocess_text(transcript)
                            tfidf_scores = calculate_tfidf(cleaned_transcript, word_lists)
                            row_result = {'company': ticker, 'year': year, 'transcript': transcript}
                            for parameter in word_lists.keys():
                                combined_tfidf = tfidf_scores.loc[word_lists[parameter], 'tfidf'].sum()
                                row_result[parameter] = combined_tfidf
                            final_results.append(row_result)
                    except Exception as e:
                        print(f"Error processing file {file_path}: {e}")
    if final_results:
        final_df = pd.DataFrame(final_results)
        return final_df
    else:
        print("No data processed.")
        return pd.DataFrame()

# Function to Plot the changes in cultural values over time

In [ ]:

def plot_cultural_values_change(final_df):
    cultural_values_columns = list(word_lists.keys())
    mean_cultural_values = final_df.groupby('year')[cultural_values_columns].mean()
    plt.figure(figsize=(12, 8))
    for value_column in cultural_values_columns:
        plt.plot(mean_cultural_values.index, mean_cultural_values[value_column], label=value_column, marker='o')
    plt.title("Change in Risk Types Per Year", fontsize=14)
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("Mean Risk Value", fontsize=12)
    plt.xticks(rotation=45)
    plt.legend(title="Risk Types")
    plt.tight_layout()
    plt.show()

In [ ]:
if __name__ == "__main__":
    dataset_path = "/kaggle/input/item-1a/ITEM 1A RISK FACTORS"
    final_df = process_yearwise_data(dataset_path, word_lists)
    if not final_df.empty:
        print(f"Processed {final_df.shape[0]} rows successfully.")
        plot_cultural_values_change(final_df)


In [ ]:
final_df.to_csv("30_risk_factors_s&p_500.csv", index=False)